In [3]:
# Title: A_Aug_Swap_Class_Using_HSV_Red_Yellow_colour_threshold.jpynb 
# Include with dissertation 
"""
Title: A_Aug_Swap_Class_Using_Red_Yellow_Colour_Detection.jpynb 

Description:
Extract red/yellow flag segments from images, refines the mask. It overlays the extracted red/yellow flag segment over the target class in the corresponding image
in the target folder (based on filename). It overlays the red/yellow flag region onto the target region and updates the annotations. 
Suitable for batch processing of images and supports mask refinement and alignment.

Features
- Generates and refines the mask for red-yellow flags.
- Resizes and overlays the mask onto the target class region. 
  Updates the Yolo format annotations for the newly created image to reflect the overlay.
- Batch processes images to apply the mask overlay.
- Handles input and output folders for source and target images.

Dependencies:
- OpenCV (cv2): For image processing.
- NumPy: For numerical operations.
- os: For file and directory management.
"""

import os
import cv2
import numpy as np

def create_merged_filename(file_name, merge_from_job, source_class_id, merge_to_job, target_class):
    # Split the filename into base and extension
    base_name, ext = os.path.splitext(file_name)
    # Replace the first part of the filename before the first underscore
    parts = base_name.split('_', maxsplit=1)
    if len(parts) == 2:
        remainder = parts[1]
        new_name = (f"Merge_RY_{merge_from_job}_into_{merge_to_job}_class_{target_class}_{remainder}{ext}")
    else:
        new_name = f"Merge_RY_{merge_from_job}_into_{merge_to_job}_class_{target_class}_{ext}"
    
    return new_name


def create_flag_mask(img_path, output_mask_path):
    """
    Creates a binary mask for red and yellow flags using HSV color thresholds.
    Applies morphological operations (closing) and Gaussian smoothing to enhance the mask.
    Refines mask edges using erosion, dilation, and additional smoothing.
    Crops the mask to the bounding box of the largest contour    
    Args:
        img_path (str): Path to the original image.
        output_mask_path (str): Path to save the intermediate mask.
    Returns:
        transparent_flag (numpy array): Image with the red-yellow mask and alpha channel.
    """
    img = cv2.imread(img_path)
    #print(f"Source Image Size: {img.shape}")  # Debugging image size
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define thresholds for red and yellow in HSV
    lower_red1, upper_red1 = np.array([0, 70, 50]), np.array([10, 255, 255])
    lower_red2, upper_red2 = np.array([160, 70, 50]), np.array([180, 255, 255])
    lower_yellow, upper_yellow = np.array([15, 70, 70]), np.array([40, 255, 255])

    # Generate masks for red and yellow
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Combine masks and refine with morphological closing
    combined_mask = cv2.bitwise_or(mask_red, mask_yellow)
    kernel = np.ones((3, 3), np.uint8)
    filled_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    #smoothed_mask = cv2.GaussianBlur(filled_mask, (5, 5), 0)
    smoothed_mask = cv2.GaussianBlur(filled_mask, (5, 5), sigmaX=1)

    # Add transparency (alpha channel)
    b, g, r = cv2.split(img)
    alpha = smoothed_mask
    transparent_flag = cv2.merge([b, g, r, alpha])

    # Save the intermediate mask for inspection
    #cv2.imwrite(output_mask_path, transparent_flag)
    #print(f"Intermediate mask saved: {output_mask_path}")

    # Crop the mask 
    # Find contours of the non-zero region
    contours, _ = cv2.findContours(transparent_flag[:, :, 3], cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        raise ValueError("No non-zero region found in the mask.")

    # Get bounding box of the largest contour
    x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))

    # Crop the mask to this bounding box
    cropped_mask = transparent_flag[y:y + h, x:x + w]
    cv2.imwrite(output_mask_path, cropped_mask)
    #print(f"Cropped mask saved: {output_mask_path}")

    return cropped_mask
    #return transparent_flag

def create_merged_filename(file_name, merge_from_job, source_class_id, merge_to_job, target_class):
    """
    Create a new filename to follow a schema 
    """
    base_name, ext = os.path.splitext(file_name)
    parts = base_name.split('_', maxsplit=1)
    if len(parts) == 2:
        remainder = parts[1]
        new_name = (f"Merge_RY_{merge_from_job}_into_{merge_to_job}_class_{target_class}_{remainder}{ext}")
    else:
        new_name = f"Merge_RY_{merge_from_job}_into_{merge_to_job}_class_{target_class}_{ext}"
    return new_name

def overlay_class_region(src_mask, tgt_image, tgt_annotations, target_class):
    """
    Resizes the refined mask to match the bounding box of a target class, preserving aspect ratio.
    Overlays the resized mask onto the specified region in the target image using alpha blending.
    """
    image_height, image_width = tgt_image.shape[:2]
    new_annotation = None

    # Crop the flag mask to its non-zero region
    # src_mask = crop_to_mask(src_mask)
    #print(f"Cropped Mask Size: {src_mask.shape}")  # Cropped mask size

    for annotation in tgt_annotations:
        class_id, x_center, y_center, width, height = annotation
        if int(class_id) == target_class:  # Match the red flag (class 0)
            # Convert normalized bounding box to pixel values
            tgt_width = int(width * image_width)
            tgt_x_center = int(x_center * image_width)
            tgt_y_center = int(y_center * image_height)

            # Preserve aspect ratio of the source flag mask
            aspect_ratio = src_mask.shape[0] / src_mask.shape[1]  # Height / Width of source mask
            resized_height = int(tgt_width * aspect_ratio)

            # Resize the mask width while preserving aspect ratio
            resized_src_flag = cv2.resize(src_mask, (tgt_width, resized_height), interpolation=cv2.INTER_LINEAR)
            print(f"Resized Mask Size: {resized_src_flag.shape}")  # Debugging resized mask size

            # Calculate top-left corner for placing the mask
            tgt_x_min = max(0, tgt_x_center - tgt_width // 2)
            tgt_y_min = max(0, tgt_y_center - resized_height // 2)

            # Overlay the mask onto the target image using alpha blending
            for c in range(3):  # B, G, R channels
                tgt_image[tgt_y_min:tgt_y_min+resized_height, tgt_x_min:tgt_x_min+tgt_width, c] = \
                    resized_src_flag[:, :, c] * (resized_src_flag[:, :, 3] > 0) + \
                    tgt_image[tgt_y_min:tgt_y_min+resized_height, tgt_x_min:tgt_x_min+tgt_width, c] * (resized_src_flag[:, :, 3] == 0)

            # Update annotation for the overlay
            new_annotation = [1, round(tgt_x_center / image_width, 6), round(tgt_y_center / image_height, 6),
                              round(tgt_width / image_width, 6), round(resized_height / image_height, 6)]
            break

    return tgt_image, new_annotation


def swap_in_class_by_colour_and_class(src_folder, tgt_folder, merge_folder, target_class, merge_from_job, merge_to_job):
    """
    Swap source class over target class for images in source and target folders, overlay the red/yellow mask, and save the results.
    """
    if not os.path.exists(merge_folder):
        os.makedirs(merge_folder)

    for file_name in os.listdir(src_folder):
        if file_name.lower().endswith('.png'):
            # Refine the red/yellow flag mask
            src_image_path = os.path.join(src_folder, file_name)
            intermediate_mask_path = os.path.join(merge_folder, f"mask_{file_name}")
            #cropped_mask_path = os.path.join(merge_folder, f"cropped_mask_{file_name}")
            flag_mask = create_flag_mask(src_image_path, intermediate_mask_path)

            # Load target image and annotations
            tgt_image_path = os.path.join(tgt_folder, file_name)
            tgt_image = cv2.imread(tgt_image_path)
            tgt_annotation_path = os.path.join(tgt_folder, os.path.splitext(file_name)[0] + ".txt")

            if not os.path.exists(tgt_annotation_path):
                #print(f"Annotation file missing for {file_name}. Skipping.")
                continue

            with open(tgt_annotation_path, 'r') as f:
                tgt_annotations = [list(map(float, line.split())) for line in f]

            # Overlay the mask onto the target bounding box
            updated_image, new_annotation = overlay_class_region(
                flag_mask, tgt_image, tgt_annotations, target_class)

            # Save updated image and annotations
            new_filename = create_merged_filename(file_name, merge_from_job, 1, merge_to_job, target_class)
            output_path = os.path.join(merge_folder, new_filename)
            cv2.imwrite(output_path, updated_image)
            with open(os.path.splitext(output_path)[0] + ".txt", 'w') as f:
                f.write(' '.join(map(str, new_annotation)) + '\n')

            #print(f"Flag class swapped in and image saved: {output_path}")

# Paths and parameters
merge_class_from = "D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Switch_class_1_from_43_into_88/Switch_flag_out_of_43"
merge_class_into = "D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Switch_class_1_from_43_into_88/Switch_flag_into_88"
merge_fldr = "D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Switch_class_1_from_43_into_88/Test"
class_to_replace = 0  # Target class (red flag)
merge_from_job = '43'
merge_to_job = '88'

# Run the process
swap_in_class_by_colour_and_class(merge_class_from, merge_class_into, merge_fldr, class_to_replace, merge_from_job, merge_to_job)

Source Image Size: (1080, 1920, 3)
Cropped Mask Size: (182, 256, 4)
Resized Mask Size: (163, 230, 4)
Source Image Size: (1080, 1920, 3)
Cropped Mask Size: (188, 271, 4)
Resized Mask Size: (162, 234, 4)
Source Image Size: (1080, 1920, 3)
Annotation file missing for source_frame_000020.PNG. Skipping.
Source Image Size: (1080, 1920, 3)
Cropped Mask Size: (214, 252, 4)
Resized Mask Size: (197, 233, 4)
Source Image Size: (1080, 1920, 3)
Cropped Mask Size: (190, 268, 4)
Resized Mask Size: (163, 230, 4)
Source Image Size: (1080, 1920, 3)
Cropped Mask Size: (187, 273, 4)
Resized Mask Size: (160, 234, 4)
Source Image Size: (1080, 1920, 3)
Cropped Mask Size: (196, 251, 4)
Resized Mask Size: (176, 226, 4)
Source Image Size: (1080, 1920, 3)
Cropped Mask Size: (183, 260, 4)
Resized Mask Size: (163, 233, 4)
Source Image Size: (1080, 1920, 3)
Cropped Mask Size: (186, 260, 4)
Resized Mask Size: (156, 219, 4)
Source Image Size: (1080, 1920, 3)
Cropped Mask Size: (185, 262, 4)
Resized Mask Size: (158, 2

AttributeError: 'NoneType' object has no attribute 'shape'